# GTA HS20 - Übung 10: Density-based Clustering zur ortsbezogenen Analyse von GPS Trajektorien

## Ziel der Übung
Das Ziel dieser Übung ist die Entwicklung / Anpassung eines Python Skripts, das es Ihnen erlaubt Haltepunkte aus Taxitrajektorien, die in Rom aufgezeichnet wurden, zu extrahieren. 
Dazu sollen sie den DBSCAN (Density-based clustering) Algorithmus von Ester, Kriegel, Sander und Xu (siehe Vorlesung) benutzen, um Bewegungsmuster in den GPS Punkten durch deren Dichte in Raum und Zeit zu clustern.

## Zum GPS Trajektorien Datensatz
Die Daten stammen aus einem öffentlich verfügbaren Datensatz bei dem 320 Taxis mit GPS Empfängern ausgestattet wurden und über einen Zeitraum von 30 Tagen getrackt wurden. 
Beim Tracking wird durchschnittlich 1 GPS Punkt alle 7 Sekunden erzeugt.
Dieser Datensatz wurde ursprünglich zur Verbesserung der Seuchenprävention verwendet, wir werden ihn heute verwenden um die Haltepunkte der Taxis automatisch zu identifizieren.

## Vorgehen

### 1) Vorüberlegung: Wie kann DBSCAN Haltepunkte erkennen
Rufen Sie sich ihr Wissen zu DBSCAN aus der Vorlesung in Erinnerung, überlegen Sie sich wie Sie einen Haltepunkt für Taxis definieren können und welche raumzeitlichen Anforderungen er erfüllen muss. 

Überlegen Sie sich Antworten auf folgende Fragen:

- Wie könnte ein sinnvoller Haltepunkt für Taxis definiert sein?
- Wie können Sie diese raumzeitlichen Anforderungen mit DBSCAN erfüllen?
- Wie beeinflussen die raumzeitlichen Anforderungen die Parameter von DBSCAN (min samples, eps, distance metric)?
- Was könnten gute Parameter sein?

Diskutieren Sie Ihre Antworten mit Ihrem Nachbarn/ Ihrer Nachbarin und notieren Sie
mögliche Lösungen.

### 2) Übersicht über Funktionen und das Programm bekommen
Hier eine kurze Übersicht über die Funktionen welche wir am Schluss aufrufen werden. Die Funktionen `transform()` und `clustering_with_dbscan()` müssen verändert werden.
- `read_data()`: Die Daten werden aus der `.csv`-Datei in eine Liste `[X, Y, T]` (x-Koordinate, y-Koordinate, Zeitstempel) eingelesen.
- `transform()`: Reprojeziert die Daten.
- `plot_nb_dists()`: Erstellt das n-nearest-neighbours Diagramm.
- `clustering_with_dbscan()`: Berechnet die Cluster mit DBSCAN.
- `plot_cluster()`: Erstellt eine Grafik um Ihnen schnell das Ergebnis zu zeigen.
- `export_to_shp()`: Exportiert die GPS Punkte mit Ihrer Clusternummer als Shapefile.

Wir beginnen zuerst mit dem Importieren einiger Bibliotheken und dem Setzen einiger Optionen.

In [ ]:
import datetime
import pyproj
import numpy as np
from sklearn.cluster import DBSCAN

from spatial_data_mining_functions import read_data, plot_nb_dists, plot_cluster, plot_cluster_interactive, export_to_shp

In [ ]:
# Path to geodatabase
input_file = r"input_data/taxi_21.txt"

# Source projection
proj_wgs84 = "EPSG:4326"

# Target protection
proj_target = "EPSG:25833"

### 3) Einlesen und Transformieren der Daten

#### Ziel: Verändern Sie die Funktion transform(), sodass die Koordinaten in ein geeignetes metrisches System projiziert werden und der Zeitstempel in kontinuierliche und interpretierbare Zeitwerte umgewandelt wird.

Die Funktion `transform()` ist noch unvollständig. 
Sie müssen die Funktion in einer Weise implementieren, so dass die Koordinaten in interpretierbaren räumlichen Distanzen vorliegen (z.B. Meter), und die Zeitwerte kontinuierliche und interpretierbare Zeitwerte (z.B. Sekunden) sind. 
Beides brauchen Sie zur Festlegung des Distanzparameters im Clustering.
Hinweis: Wenn zwei `datetime.datetime` Objekte subtrahiert werden, wird ein `datetime.timedelta` Objekt zurückgegeben, kein kontinuierlicher Wert.

Mittels der Bibliothek `pyproj` können sie lon / lat Koordinaten in eine andere Projektion transformieren. Benutzen Sie dazu den `Transformer` (https://pyproj4.github.io/pyproj/stable/examples.html#step-2-create-transformer-to-convert-from-crs-to-crs). U.U. müssen Sie die Option `always_xy=True` setzen, da ansonsten die Koordinaten "umgedreht" sind.

In [ ]:
def transform(data_in):
    """Function that transforms data to metric coordinate system and continuous timestamps."""
    data_out = []
    t_reference = datetime.datetime(2014, 1, 1)
    
    # TODO Initialize transformer accordingly.
    transformer = pyproj.Transformer()
    
    # Iterate over every point in the input data.
    for d in data_in:
        x = d[0]
        y = d[1]
        ts = d[2]
        
        # TODO Replace this with according transformations.
        x, y = 0, 0
        ts = 0
        
        data_out.append([x, y, ts])
    return data_out

### 4) Implementierung von DBSCAN

#### Ziel: Verändern Sie die Funktion `clustering_with_dbscan()`, sodass sie die Cluster Labels und die Indizies der Core Samples berechnet und ausgibt.

- Importieren Sie den DBSCAN Algorithmus aus Scikit-learn.
- Initialisieren Sie ein DBSCAN Objekt (Übergeben Sie dabei die Input Daten).
- Nutzen Sie die `fit()` Methode des DBSCAN Objekts um ein Clustering zu berechnen.
- Weisen Sie das Ergebnis des Clusterings den Variablen `labels` und `core_samples_indices` zu.

Hilfestellungen:

- Offizielle Dokumentation: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
- Beispiel: https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-autoexamples-cluster-plot-dbscan-py

In [ ]:
def clustering_with_dbscan(X, eps=1, min_samples=2, metric='cityblock'):
    """ Function derived from scipy dbscan example
    http://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#example-cluster-plot-dbscan-py."""

    X = np.array(data)

    # TODO: Compute DBSCAN
    db = DBSCAN()
    
    # TODO: Assign proper labels and core_sample_indices to labels.
    labels = np.random.randint(0, 5, size=len(data))
    core_samples_indices = [1,]
    
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    print('Estimated number of clusters: %d' % n_clusters_)
    return labels, core_samples_indices

In [ ]:
# Read data, limit number of rows for speed (you can try more later).
org_data = read_data(input_file, nrows=2000)

# Apply transformations.
data = transform(org_data)

# Plot nearest neighbor distances diagram.
metric = 'cityblock'
data = np.array(data)
plot_nb_dists(data, nearest_neighbor=[7, 10, 15], metric=metric, ylim=250)

# Calculate clusters with dbscan.
eps = 1
min_samples = 2
labels, core_samples_indices = clustering_with_dbscan(data, eps=eps, min_samples=min_samples, metric=metric)

# Export clusters to geodatabase.
export_layer_name = "stops"
export_to_shp(data, labels, export_layer_name, crs=proj_target.srs)

# Plot clusters. Use the first function to plot a static map.
# plot_cluster(data, labels, core_samples_indices, proj_wgs84=proj_wgs84, proj_target=proj_target, linestyle='solid')
plot_cluster_interactive(data, labels, core_samples_indices, proj_wgs84=proj_wgs84, proj_target=proj_target)

### 5) Wahl der Parameter

Der Code und das Clustering sind jetzt voll Funktionsfähig. 
Das Ergebnis hängt allerdings stark von den gewählten Parametern ab. 
Ziel ist nun, die Parameter so anzupassen dass plausible Cluster generiert werden. 

Sie müssen als erstes eine Metrik finden, die es ihnen erlaubt den Distanzradius (`eps`) Parameter von DBSCAN (siehe auch Vorlesung) über alle Dimensionen von Raum und Zeit zu bestimmen, sowie den Punktdichteparameter (`min_samples`).

- Überlegen sie, wie man Raum und Zeit in einer gemeinsamen Distanzmetrik darstellen könnte. 
Gehen sie dazu durch die [Liste der erlaubten Distanz Metriken](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html) und wählen sie eine aus, die es ihnen auf einfachste Weise erlaubt eine maximale Distanz auf den drei vorhandenen Dimensionen (X, Y, Zeit) zu definieren. 
Nutzen sie Wikipedia um unbekannte Distanzmetriken für ihre Eignung zu beurteilen.
Tipp: Eine der folgenden Metriken kommt in Frage: City Block, Euklidisch, Chebyshev. Aufgrund welcher Überlegung können sie diese Wahl treffen?
- Zum Bestimmen von `eps` und `min_samples`: Nutzen Sie die Funktion `plot_nb_dists` um einen Summen-Plot zu generieren, der den Anteil der Punkte (x-Achse) gegen die Distanz zum k-ten nächsten Nachbar (y-Achse) aufträgt (siehe Vorlesung).
Dabei können sie die Punkte zum Beispiel nach ihrer Distanz zum 4-nächsten Nachbar sortieren. 
Identifizieren Sie aus dem Plot eine geeignete Distanzgrenze um die Bewegungsarten schnell / langsam zu unterscheiden.
- Wählen sie einen maximalen Radius einer Nachbarschaft (`eps`) aus. 
Das ist eine Raum-Zeit Distanz, mithilfe derer sie die Nachbarschaft eines Punktes definieren.
- Wählen sie eine entsprechende minimale Anzahl der Nachbarn (die minimale Dichte; `min_samples`) für die gewählte Nachbarschaft aus (zum Beispiel 4 bei einem Plot zum 4-nächsten Nachbar).

Die Parameter können sie im Laufe der Übung noch verändern und anpassen, um plausiblere Cluster zu generieren

    
### 6) Ergebnisse Visualisieren und Parameter Tunen

Passen sie die Parameter der `clustering_with_dbscan()` Funktion (`eps`, `min_samples`, `metric`) an und führen Sie sie aus.
Plotten sie die gefunden Cluster mit der Funktion `plot_cluster()` oder `plot_cluster_interactive()`.
Schauen sie sich den Konsolen output und die Cluster im Plot an. 
Das Ergebnis der Methode sind die Clusterlabels für jeden Punkt des Trajektoriendatensatzes. 
Wie liest man aus dem Clusteringergebnis Clusterlabels und Core Points? 
Lassen sie sich das Ergebnis in der Konsole ausgeben und schauen Sie sich die Cluster Labels an. 
Ein Label von `-1` bedeutet "Noise".
    
### 7) Ergebnis exportieren und auf Karte visualisieren

Exportieren sie die gefunden Cluster mittels `export_to_shp()` als Shapefile. 
Sehen Sie sich das Clusterergebnis als einen Layer in ArcGIS / QGIS mit der Hintergrundkarte an (färben
Sie die Punkte nach Clusterzugehörigkeit ein).